In [1]:
import numpy as np 
import pandas as pd

from pytorch_tabnet.metrics import Metric
from pytorch_tabnet.tab_model import TabNetRegressor

import torch
from torch.optim import Adam, SGD
from torch.optim.lr_scheduler import ReduceLROnPlateau, CosineAnnealingWarmRestarts

from scipy import stats
from scipy.stats import norm
from sklearn.preprocessing import MinMaxScaler,LabelEncoder, StandardScaler
from tqdm.auto import tqdm

target_name = 'target'

In [2]:
train = pd.read_pickle('input/train_248.pkl')
test = pd.read_pickle('input/test_247.pkl')

In [3]:
X = train.drop(['row_id','target', 'time_id'], axis = 1)
y = train['target']
X_test=test.copy()
X_test.drop(['time_id','row_id'], axis=1,inplace=True)

In [4]:
X

,stock_id,wap1_sum,wap1_std,wap2_sum,wap2_std,wap3_sum,wap3_std,wap4_sum,wap4_std,log_return1_realized_volatility,...,trade_order_count_sum_0c1,trade_order_count_sum_1c1,trade_order_count_sum_3c1,trade_order_count_sum_4c1,trade_order_count_sum_6c1,size_tau2_0c1,size_tau2_1c1,size_tau2_3c1,size_tau2_4c1,size_tau2_6c1
0,0,303.125061,0.000693,303.105530,0.000781,303.134857,0.000637,303.146942,0.000652,0.004499,...,413.684211,555.200000,235.550000,519.90625,419.5,0.058550,0.057267,0.078471,0.054691,0.050700
1,0,200.047775,0.000262,200.041168,0.000272,200.035614,0.000298,200.040848,0.000278,0.001204,...,244.578947,225.000000,126.900000,212.59375,481.0,0.081235,0.078955,0.122289,0.078616,0.045740
2,0,187.913849,0.000864,187.939819,0.000862,187.923065,0.000670,187.897369,0.000779,0.002369,...,236.842105,193.266667,125.050000,246.50000,155.5,0.078550,0.087378,0.116278,0.074977,0.080722
3,0,119.859779,0.000757,119.835945,0.000656,119.870163,0.000606,119.884239,0.000733,0.002574,...,166.736842,196.766667,116.200000,167.75000,450.5,0.100382,0.089673,0.105948,0.094684,0.055447
4,0,175.932861,0.000258,175.934250,0.000317,175.928284,0.000215,175.912537,0.000312,0.001894,...,183.157895,182.433333,101.947368,178.65625,465.5,0.087285,0.089068,0.112663,0.086381,0.046358
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
428927,126,309.870453,0.000486,309.871368,0.000613,309.778625,0.000596,309.812317,0.000594,0.003691,...,328.947368,354.733333,175.250000,324.93750,619.0,0.064380,0.064007,0.089665,0.061596,0.041157
428928,126,223.552139,0.001264,223.580322,0.001303,223.505493,0.001340,223.480896,0.001341,0.004104,...,335.842105,362.600000,186.950000,352.00000,262.0,0.073644,0.065909,0.085298,0.062453,0.062479
428929,126,256.277039,0.000466,256.255066,0.000599,256.146057,0.000520,256.168884,0.000558,0.003117,...,183.684211,216.300000,111.100000,184.06250,87.5,0.098654,0.092117,0.124247,0.090574,0.120507
428930,126,399.721741,0.000456,399.714325,0.000507,399.775391,0.000424,399.779358,0.000429,0.003661,...,544.947368,451.233333,262.650000,496.28125,690.0,0.051648,0.054263,0.067649,0.051358,0.040747


In [5]:
X_test

,stock_id,wap1_sum,wap1_std,wap2_sum,wap2_std,wap3_sum,wap3_std,wap4_sum,wap4_std,log_return1_realized_volatility,...,trade_order_count_sum_0c1,trade_order_count_sum_1c1,trade_order_count_sum_3c1,trade_order_count_sum_4c1,trade_order_count_sum_6c1,size_tau2_0c1,size_tau2_1c1,size_tau2_3c1,size_tau2_4c1,size_tau2_6c1
0,0,3.001215,0.00017,3.001650,0.000153,3.000752,0.000142,2.999481,0.000017,0.000294,...,412.498364,11.00000,192.853226,406.295771,549.789763,0.069893,0.301511,0.093019,0.067442,0.058326
1,0,389.911957,0.00111,389.912109,0.001149,389.913208,0.001104,389.915283,0.001123,0.004233,...,412.498364,408.32996,192.853226,406.295771,549.789763,0.069893,0.069602,0.093019,0.067442,0.058326
2,0,389.911957,0.00111,389.912109,0.001149,389.913208,0.001104,389.915283,0.001123,0.004233,...,412.498364,408.32996,192.853226,406.295771,549.789763,0.069893,0.069602,0.093019,0.067442,0.058326


In [6]:
nunique = X.nunique()
types = X.dtypes

categorical_columns = []
categorical_dims =  {}

for col in X.columns:
    if  col == 'stock_id':
        l_enc = LabelEncoder()
        X[col] = l_enc.fit_transform(X[col].values)
        X_test[col] = l_enc.transform(X_test[col].values)
        categorical_columns.append(col)
        categorical_dims[col] = len(l_enc.classes_)
    else:
        scaler = StandardScaler()
        X[col] = scaler.fit_transform(X[col].values.reshape(-1, 1))
        X_test[col] = scaler.transform(X_test[col].values.reshape(-1, 1))
        
cat_idxs = [ i for i, f in enumerate(X.columns.tolist()) if f in categorical_columns]
cat_dims = [ categorical_dims[f] for i, f in enumerate(X.columns.tolist()) if f in categorical_columns]

In [7]:
X

,stock_id,wap1_sum,wap1_std,wap2_sum,wap2_std,wap3_sum,wap3_std,wap4_sum,wap4_std,log_return1_realized_volatility,...,trade_order_count_sum_0c1,trade_order_count_sum_1c1,trade_order_count_sum_3c1,trade_order_count_sum_4c1,trade_order_count_sum_6c1,size_tau2_0c1,size_tau2_1c1,size_tau2_3c1,size_tau2_4c1,size_tau2_6c1
0,0,-0.639031,-0.396657,-0.639172,-0.345354,-0.638971,-0.442527,-0.638893,-0.443297,0.074239,...,0.005834,0.687678,0.494129,0.474315,-0.239017,-0.937352,-1.065469,-0.909015,-0.998361,-0.354560
1,0,-1.397873,-0.806591,-1.397917,-0.823158,-1.397975,-0.764148,-1.397950,-0.795321,-0.844605,...,-0.826092,-0.858391,-0.763276,-0.808691,-0.126195,0.937313,0.807846,1.828837,0.874842,-0.585207
2,0,-1.487201,-0.233758,-1.487005,-0.268987,-1.487146,-0.411477,-1.487349,-0.323818,-0.519951,...,-0.864154,-1.006974,-0.784686,-0.667135,-0.723326,0.715387,1.535442,1.453243,0.589960,1.041356
3,0,-1.988207,-0.336161,-1.988376,-0.462645,-1.988144,-0.472623,-1.988055,-0.367268,-0.462750,...,-1.209042,-0.990586,-0.887107,-0.995910,-0.182147,2.519582,1.733664,0.807853,2.132899,-0.133880
4,0,-1.575404,-0.810563,-1.575389,-0.781568,-1.575450,-0.842968,-1.575580,-0.763420,-0.652201,...,-1.128257,-1.057698,-1.052052,-0.950378,-0.154630,1.437222,1.681385,1.227384,1.482879,-0.556482
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
428927,111,-0.589373,-0.593528,-0.589363,-0.503068,-0.590060,-0.482083,-0.589823,-0.497965,-0.151262,...,-0.411035,-0.250951,-0.203722,-0.339665,0.126966,-0.455564,-0.483286,-0.209556,-0.457703,-0.798284
428928,111,-1.224837,0.146578,-1.224625,0.144222,-1.225193,0.223623,-1.225387,0.205770,-0.036078,...,-0.377116,-0.214118,-0.068318,-0.226681,-0.527951,0.309971,-0.319030,-0.482442,-0.390637,0.193093
428929,111,-0.983920,-0.613049,-0.984078,-0.516126,-0.984897,-0.553339,-0.984741,-0.531594,-0.311121,...,-1.125668,-0.899127,-0.946129,-0.927807,-0.848072,2.376775,1.944703,1.951173,1.811113,2.891248
428930,111,0.072101,-0.621811,0.072048,-0.602985,0.072485,-0.644706,0.072506,-0.653458,-0.159531,...,0.651593,0.200883,0.807757,0.375683,0.257216,-1.507694,-1.324948,-1.585180,-1.259345,-0.817356


In [8]:
def rmspe(y_true, y_pred):
    # Function to calculate the root mean squared percentage error
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))

class RMSPE(Metric):
    def __init__(self):
        self._name = "rmspe"
        self._maximize = False

    def __call__(self, y_true, y_score):
        
        return np.sqrt(np.mean(np.square((y_true - y_score) / y_true)))
    

def RMSPELoss(y_pred, y_true):
    return torch.sqrt(torch.mean( ((y_true - y_pred) / y_true) ** 2 )).clone()
tabnet_params = dict(
    cat_idxs=cat_idxs,
    cat_dims=cat_dims,
    cat_emb_dim=1,
    n_d = 32, # 16
    n_a = 32, # 16
    n_steps = 2,
    gamma = 2,
    n_independent = 2,
    n_shared = 2,
    lambda_sparse = 0, 
    optimizer_fn = Adam, 
    optimizer_params = dict(lr = (2e-2)),
    mask_type = "entmax",
    scheduler_params = dict(T_0=200, T_mult=1, eta_min=1e-4, last_epoch=-1, verbose=False),
    scheduler_fn = CosineAnnealingWarmRestarts,
    seed = 42,
    verbose = 5
)
# kfold = KFold(n_splits = 5, random_state = 42, shuffle = True)
# # Create out of folds array
# oof_predictions = np.zeros((X.shape[0], 1))
# test_predictions_tabnet = np.zeros(X_test.shape[0])
# feature_importances = pd.DataFrame()
# feature_importances["feature"] = X.columns.tolist()
# # stats = pd.DataFrame()
# explain_matrices = []
# masks_ =[]

# for fold, (trn_ind, val_ind) in enumerate(kfold.split(X)):
#     print(f'Training fold {fold + 1}')
#     X_train, X_val = X.iloc[trn_ind].values, X.iloc[val_ind].values
#     y_train, y_val = y.iloc[trn_ind].values.reshape(-1,1), y.iloc[val_ind].values.reshape(-1,1)

#     clf =  TabNetRegressor(**tabnet_params)
#     clf.fit(
#       X_train, y_train,
#       eval_set=[(X_val, y_val)],
#       max_epochs = 300,
#       patience = 50,
#       batch_size = 1024*10, 
#       virtual_batch_size = 128*10,  # ?
#       num_workers = 4,
#       drop_last = False,
#       eval_metric=[RMSPE],
#       loss_fn=RMSPELoss
#       )
    
#     saving_path_name = f"./fold{fold}"
#     saved_filepath = clf.save_model(saving_path_name)
    
#     explain_matrix, masks = clf.explain(X_val)
#     explain_matrices.append(explain_matrix)
#     masks_.append(masks[0])
#     masks_.append(masks[1])
      
#     oof_predictions[val_ind] = clf.predict(X_val)
#     test_predictions_tabnet+=clf.predict(X_test.values).flatten()/5
#     feature_importances[f"importance_fold{fold}+1"] = clf.feature_importances_
    
# #     stats[f'fold{fold+1}_train_rmspe']=clf.history['loss']
# #     stats[f'fold{fold+1}_val_rmspe']=clf.history['val_0_rmspe']
    
# print(f'OOF score across folds: {rmspe(y, oof_predictions.flatten())}')

In [9]:
# del X_train, X_val,y_train,y_val
# gc.collect()

## Load Tabnet_models

In [10]:
# import os
# import zipfile
# startdir = "../input/tabnet-models-32/fold0"  #要压缩的文件夹路径，这里选择将input中的所有文件压缩
# file_news = 'tabnet32_f0.zip' # 压缩后文件夹的名字，这里压缩到kaggle之中的output文件之中，名称为result.zip
# z = zipfile.ZipFile(file_news,'w',zipfile.ZIP_DEFLATED) #参数一：文件夹名
# for dirpath, dirnames, filenames in os.walk(startdir):
#     fpath = dirpath.replace(startdir,'') #这一句很重要，不replace的话，就从根目录开始复制
#     fpath = fpath and fpath + os.sep or ''#实现当前文件夹以及包含的所有文件的压缩
#     for filename in filenames:
#         z.write(os.path.join(dirpath, filename),fpath+filename)
#         print ('压缩成功')
# z.close()

# startdir = "../input/tabnet-models-32/fold1"  #要压缩的文件夹路径，这里选择将input中的所有文件压缩
# file_news = 'tabnet32_f1.zip' # 压缩后文件夹的名字，这里压缩到kaggle之中的output文件之中，名称为result.zip
# z = zipfile.ZipFile(file_news,'w',zipfile.ZIP_DEFLATED) #参数一：文件夹名
# for dirpath, dirnames, filenames in os.walk(startdir):
#     fpath = dirpath.replace(startdir,'') #这一句很重要，不replace的话，就从根目录开始复制
#     fpath = fpath and fpath + os.sep or ''#实现当前文件夹以及包含的所有文件的压缩
#     for filename in filenames:
#         z.write(os.path.join(dirpath, filename),fpath+filename)
#         print ('压缩成功')
# z.close()

# startdir = "../input/tabnet-models-32/fold2"  #要压缩的文件夹路径，这里选择将input中的所有文件压缩
# file_news = 'tabnet32_f2.zip' # 压缩后文件夹的名字，这里压缩到kaggle之中的output文件之中，名称为result.zip
# z = zipfile.ZipFile(file_news,'w',zipfile.ZIP_DEFLATED) #参数一：文件夹名
# for dirpath, dirnames, filenames in os.walk(startdir):
#     fpath = dirpath.replace(startdir,'') #这一句很重要，不replace的话，就从根目录开始复制
#     fpath = fpath and fpath + os.sep or ''#实现当前文件夹以及包含的所有文件的压缩
#     for filename in filenames:
#         z.write(os.path.join(dirpath, filename),fpath+filename)
#         print ('压缩成功')
# z.close()

# startdir = "../input/tabnet-models-32/fold3"  #要压缩的文件夹路径，这里选择将input中的所有文件压缩
# file_news = 'tabnet32_f3.zip' # 压缩后文件夹的名字，这里压缩到kaggle之中的output文件之中，名称为result.zip
# z = zipfile.ZipFile(file_news,'w',zipfile.ZIP_DEFLATED) #参数一：文件夹名
# for dirpath, dirnames, filenames in os.walk(startdir):
#     fpath = dirpath.replace(startdir,'') #这一句很重要，不replace的话，就从根目录开始复制
#     fpath = fpath and fpath + os.sep or ''#实现当前文件夹以及包含的所有文件的压缩
#     for filename in filenames:
#         z.write(os.path.join(dirpath, filename),fpath+filename)
#         print ('压缩成功')
# z.close()

# startdir = "../input/tabnet-models-32/fold4"  #要压缩的文件夹路径，这里选择将input中的所有文件压缩
# file_news = 'tabnet32_f4.zip' # 压缩后文件夹的名字，这里压缩到kaggle之中的output文件之中，名称为result.zip
# z = zipfile.ZipFile(file_news,'w',zipfile.ZIP_DEFLATED) #参数一：文件夹名
# for dirpath, dirnames, filenames in os.walk(startdir):
#     fpath = dirpath.replace(startdir,'') #这一句很重要，不replace的话，就从根目录开始复制
#     fpath = fpath and fpath + os.sep or ''#实现当前文件夹以及包含的所有文件的压缩
#     for filename in filenames:
#         z.write(os.path.join(dirpath, filename),fpath+filename)
#         print ('压缩成功')
# z.close()

In [11]:
# 5 models -> valid
model_path='models/'
valid_preds_tabnet = np.zeros(len(X))
for i in range(0,5):
    loaded_clf = TabNetRegressor()
    loaded_clf.load_model(model_path + 'tabnet_models/fold{}.zip'.format(i))
    valid_preds_tabnet += loaded_clf.predict(X.values).reshape(-1)/5
    
print('tabnet_32 -> ', rmspe(y, valid_preds_tabnet))

Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
tabnet_32 ->  0.20066398946434455


In [12]:
# 5 models -> predict
test_preds_tabnet = np.zeros(len(X_test))
for i in range(0,5):
    loaded_clf = TabNetRegressor()
    loaded_clf.load_model(model_path + 'tabnet_models/fold{}.zip'.format(i))
    test_preds_tabnet += loaded_clf.predict(X_test.values).reshape(-1)/5

Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda


In [13]:
pd.Series(valid_preds_tabnet).to_csv('outputs/valid_pred_tabnet.csv',index=0)

In [14]:
pd.Series(test_preds_tabnet).to_csv('outputs/test_pred_tabnet.csv',index=0)